# Demonstration of our algorithm running over a sampled instance from Gaussian process with one dimensional input.

## import relevant packages

In [3]:
import sys
sys.path.append('..')
import POP_BO.popbo.util as util
from POP_BO.popbo import POPBO
import POP_BO.popbo.Oracles as sigma
import numpy as np
from matplotlib import pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Construct the problem configurations

In [4]:
np.random.seed(3)
config = util.get_config('GP_sample_func', problem_dim=1)
popbo = POPBO(config, y_lb=-5, y_ub=5)
f = config['obj']

[[ 2.53142993]
 [ 0.31744681]
 [ 0.2812075 ]
 [ 2.86324211]
 [-0.50206966]
 [-0.52029318]
 [-1.32332505]
 [-0.30103314]
 [-2.55538644]
 [ 2.41711339]]
  index  |  rbf.lengthscale  |  constraints  |  priors
  [0]    |       1.00000000  |      +ve      |        


## Initialize the algorithm

In [5]:
x_prime = [np.random.rand()]
x = popbo.query_new_point(x_prime=x_prime)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

[7.16844106] with opt improvement 6.348864544866224, real 6.3488647186567615


## repeatedly and interatively run the following block to see how our algorithm works

In [8]:
plot_uncer = True  # set to false if you do not want to plot to save time
sample_per_run = 5
for k in range(sample_per_run):
    x_prime = x
    x = popbo.query_new_point(x_prime=x_prime)
    one = sigma.pref_oracle(f(x), f(x_prime))
    popbo.add_new_data(x_prime=x_prime, x=x, one=one)
    print(f'New sampled point: {x}, with objective value {f(x)}.')

if plot_uncer:
    num_data = popbo.num_data
    x_grid = popbo.config['x_grid']
    max_ll_list = []
    upb_list = []
    lpb_list = []
    f_list = []
    for x in x_grid:
        max_ll_list.append(popbo.get_maxLL_min_norm_inter(x))
        upb_list.append(popbo.get_upper_bound(x, beta=0.2*np.sqrt(num_data)))
        lpb_list.append(popbo.get_lower_bound(x, beta=0.2*np.sqrt(num_data)))
        f_list.append(f(x))
    sampled_f_list = []
    for x in popbo.x_list:
        sampled_f_list.append(f(x))
    
    plt.figure()
    plt.plot(x_grid, f_list)
    plt.plot(x_grid, max_ll_list)
    plt.plot(x_grid, upb_list)
    plt.plot(x_grid, lpb_list)
    plt.scatter(popbo.x_list, sampled_f_list, marker='x')
    plt.legend(['Ground truth', 'Maximum Likelihood Estimator', 'Confidence Upper Bound', 'Confidence Lower Bound', 'Sampled Points'])
    plt.show()



[5.32908765] with opt improvement 6.844730189067144, real 6.013185127015616
Estimated best x: [5.329163634540852],  with regret 0.10829738446785653.
Step 6: norm 1.4964416791857684, LL value -0.48466097846808687,avg -0.08077682974468114.
New sampled point: [5.329087653576014], with objective value 2.7303535643264305.
[5.89481655] with opt improvement 0.6091815834162646, real 0.6091819730871784
Estimated best x: [5.329087653576014],  with regret 0.10822908886453764.
Step 7: norm 1.4964416800800802, LL value -0.8008424641059406,avg -0.11440606630084867.
New sampled point: [5.894816547316576], with objective value 1.9126305079945833.
[4.85734301] with opt improvement 2.4303005077489104, real 2.4303007351477133
Estimated best x: [5.329087653576014],  with regret 0.10822908886453764.
Step 8: norm 1.4964416806095773, LL value -1.0069720781829261,avg -0.12587150977286576.
New sampled point: [4.85734301005086], with objective value 2.846315734234312.
[8.37699631] with opt improvement -0.583208

 /var/folders/vk/38fbwqvx4dg05k8q97qj8qfm0000gn/T/ipykernel_18121/1013354481.py:33: UserWarning:FigureCanvasAgg is non-interactive, and thus cannot be shown


## plot the regret evolution

In [9]:
x_list = [popbo.x_prime_list[0]] + popbo.x_list
regret = [popbo.config['opt_val'] - f(x) for x in x_list]
plt.plot(np.cumsum(regret))
plt.plot([0, popbo.num_data], [regret[0], sum(regret)])
plt.legend(['cumulative regret', 'linear reference'])
plt.show()
#plt.plot(opbo.estimated_best_x_list)

 /var/folders/vk/38fbwqvx4dg05k8q97qj8qfm0000gn/T/ipykernel_18121/3692594107.py:6: UserWarning:FigureCanvasAgg is non-interactive, and thus cannot be shown
